#**Lab.13 | IBM3202 (2021)** – Combinando Análisis Coevolutivo de Información de Secuencia y Modelos basados en Estructura para Predecir Estructuras de Proteínas







### Aspectos Teóricos

La conservación de la estructura tridimensional entre proteínas homólogas impone restricciones a la variabilidad de la secuencia. Incluso cuando la identidad de secuencia entre miembros homólogos de una familia de proteínas dada puede estar por debajo de lo que se conoce como la "zona crepuscular" (20-30% de identidad de secuencia), sus estructuras a menudo exhiben **interacciones entre pares de residuos que, cuando se mapean en su secuencia de proteína, ocupan posiciones equivalentes entre todas las proteínas**. Mientras que la naturaleza química de la interacción entre estos pares de residuos puede variar de una proteína a otra, el contacto físico entre estos residuos está **restringido en la estructura final**.

Basándose en esto, se sugirió que el **análisis estadístico de patrones correlacionados de sustitución de aminoácidos en diferentes posiciones de secuencia de una familia de proteínas** podría ser explotado para **inferir contactos espaciales dentro de la estructura terciaria de la proteína**. La hipótesis detrás de esto es bastante simple: si dos residuos están interactuando en una estructura de proteína y ocurre una mutación desestabilizadora en uno de los residuos, **una mutación compensatoria en una posición específica de la secuencia preserva la estabilidad de la arquitectura y función de la proteína (FIG 1)**.




Gracias a la explosión de la secuenciación genómica, que ha permitido la deposición de datos de **miles de secuencias de proteínas** en bases de datos de acceso público, estas firmas estadísticas dejadas por la evolución de las proteínas pueden ser utilizadas para predecir qué pares de residuos deberían estar en proximidad espacial en el plegamiento nativo funcional de la proteína.


<figure>
  <center>
    <img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/sbmdca_01.png' width=700/>
    <figcaption><b>FIGURA 1. </b>Representación esquemática de los contactos entre pares de residuos que pueden inferirse del análisis coevolutivo de secuencias lineales de proteínas.<br>El lado izquierdo de la imagen representa un alineamiento de múltiples secuencias de la misma proteína de muchos organismos. Las columnas enmarcadas indican dos posiciones en este alineamiento cuyos residuos son siempre complementarios entre sí, a pesar de las mutaciones que ocurren en cada posición. Este patrón sugiere que estas dos posiciones probablemente forman una interacción física crucial para la estabilidad de la estructura de la proteína, como se muestra en el lado derecho.<br>
    Dibujo de Sergey Ovchinikov, <i><a href="http://site.solab.org/home">http://site.solab.org/home</a></i></figcaption>
  </center>
</figure>


Estas interacciones físicas inferidas estadísticamente son **un compañero natural para los Modelos basados en Estructura (SBM)**. Los SBM canónicos **incluyen explícitamente un sesgo nativo en la función de energía potencial**, al tratar todos los pares de residuos **en contacto en la estructura nativa (FIGURA 2)** (dado un umbral de distancia) como **interacciones atractivas (FIGURA 3)**. Todas las interacciones no nativas (es decir, aquellos pares de residuos con distancias que caen fuera del umbral de distancia) se tratan como repulsivas. Esta condición es coherente con la noción de una secuencia de una proteína sin interacciones altamente frustradas, o el llamado **paisaje de plegamiento en embudo**.


<figure>
  <center>
    <img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_01.png'/>
    <figcaption><b>FIGURA 2.</b> Utilizando un criterio basado en la distancia para extraer contactos nativos de una estructura de proteína. El panel izquierdo muestra la proximidad del contacto atómico más cercano para cada par de residuos en la proteína ribosomal S6 hasta un máximo de 1.5 nm, mientras que el panel derecho muestra el mapa de contactos nativos de grano grueso resultante al filtrar por un umbral de distancia de 0.6 nm y una separación mínima de secuencia de 3 residuos entre residuos interactivos.<br> Noel JK & Onuchic JN (2012) <i>Computational modeling of biological systems, 31-54</i></figcaption>
  </center>
</figure>


Estas interacciones atractivas suelen modelarse como potenciales de Lennard-Jones o Gaussianos, siendo este último correspondiente a:

$$C_G(r_{ij},r^{ij}_0) = \left(1+\left(\frac{\sigma_{NC}}{r_{ij}}\right)^{12}\right)\left(1+G(r_{ij},r^{ij}_0)\right)-1;$$

donde $$G(r_{ij},r^{ij}_0) = -\exp \left[-(r_{ij}-r^{ij}_0)^2(2\sigma^2)\right]$$

<figure>
  <img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/sbmdca_03.png' width=500/>
  <figcaption><b>FIGURA 3.</b> Comparación de potenciales de contacto Lennard–Jones (LJ) y Gaussianos. Las curvas negras muestran potenciales de contacto LJ con mínimos a 6Å y 10Å. El potencial de contacto Gaussiano mostrado en rojo tiene un volumen excluido $NC$ que se puede establecer de forma independiente a la ubicación del mínimo. La línea roja punteada muestra cómo cambiaría el contacto Gaussiano al agregar otro mínimo a 10Å.<br> Noel JK & Onuchic JN (2012) <i>Computational modeling of biological systems, 31-54</i></figcaption>
</figure>


SBM reduce aún más la complejidad del sistema biomolecular mediante una **aproximación de grano grueso**, en la cual cada residuo se reemplaza por una cuenta centrada en las coordenadas del carbono alfa. La disminución en la granularidad del sistema, junto con su función de energía simplificada, permite simular eficientemente el plegamiento de proteínas con recursos computacionales mínimos.

En este contexto, resulta sencillo **sustituir los contactos nativos basados en la distancia extraídos de una estructura experimental por aquellos predichos por el análisis coevolutivo de secuencias de proteínas**, con el fin de utilizar SBM junto con DCA para la **predicción de la estructura de proteínas**.


##Generalidades experimentales

En este tutorial, ejemplificaremos cómo podemos **inferir contactos nativos a partir del análisis coevolutivo de información de secuencia de proteínas mediante Análisis de Acoplamiento Directo (DCA)** y cómo podemos luego **incorporar esta información en Modelos Basados en Estructura (SBM)** para predecir estructuras de proteínas.

Nos centraremos especialmente en comparar las interacciones de pares de residuos inferidas y experimentales de una **miohemeritrina** del gusano marino *Themiste hennahi*. Las proteínas hemeritrina ([**familia Pfam PF01814**](https://pfam.xfam.org/family/PF01814)) son responsables de la unión y transporte de oxígeno en invertebrados marinos y, contrario a lo esperado, no contienen un grupo hemo (como en la hemoglobina). Estas proteínas son bastante pequeñas (menos de 150 residuos de aminoácidos), lo cual es perfecto para nuestro breve tutorial.

Para el análisis coevolutivo de secuencias lineales de proteínas, emplearemos el método **DCA**. Los puntajes DCA proporcionan información cuantitativa sobre la existencia de contactos físicos en la estructura tridimensional de una biomolécula, con pares de mayor puntaje prediciendo con precisión contactos nativos que se pueden observar en estructuras de proteínas resueltas experimentalmente.

Las interacciones de pares de residuos con mayor puntaje inferidas por DCA se incorporarán como **Interacciones Gaussianas** (FIGURA 3) en los modelos **SBM**, y se combinarán con parámetros de estructura secundaria generados en base a predicciones de secuencia utilizando **Jpred4**, para plegar y predecir la estructura tridimensional de la mihohemeritrina.

Este tutorial se basa en tres fuentes de información diferentes que recomendamos explorar en mayor detalle:

- Un tutorial sobre pyDCA de Mehari B. Zerihun (disponible en [este GitHub](https://github.com/KIT-MBS/pydca)), asociado con la publicación:
  -  Zerihun, M. B., Pucci, F., Peter, E. K., & Schug, A. (2020). pydca v1. 0: a comprehensive software for Direct Coupling Analysis of RNA and Protein Sequences. *Bioinformatics, 36*(7), 2264-2265.
- Un tutorial sobre SBM de Jeff K. Noel y Paul C. Whitford (disponible en el [servidor Structure-based Models On GROMACS [SMOG]](https://smog-server.org)), con su uso y una descripción general detallada en la publicación:
  - Noel, J. K., & Onuchic, J. N. (2012). The many faces of structure-based potentials: from protein folding landscapes to structural characterization of complex biomolecules. En *Computational Modeling of Biological Systems*, 31-54; Springer, Boston, MA.
- Un tutorial y scripts de Python sobre la combinación de SBM y DCA de Ricardo N. Dos Santos (disponible en el [sitio web del Laboratorio Morcos](http://morcoslaboratory.org)), asociado con la publicación:
  -  Dos Santos, R. N., Jiang, X., Martínez, L., & Morcos, F. (2019). Coevolutionary Signals and Structure-Based Models for the Prediction of Protein Native Conformations. *Methods Mol Biol, 1851*, 83-103.


#Parte 0. Descargando e instalando el software necesario

Para llevar a cabo este tutorial, primero debemos instalar varios programas. En particular:

- **biopython**: para la manipulación de archivos PDB.
- **py3Dmol**: para la visualización de la estructura de proteínas.
- **HMMER**: para alinear múltiples secuencias de proteínas utilizando el Modelo Oculto de Markov (HMM) de conservación específica de posición.
- **pyDCA**: para predecir contactos nativos mediante el Análisis de Acoplamiento Directo.
- **Versión mejorada de GROMACS con SBM**: que contiene potenciales de contacto Gaussianos.

A continuación, se presentan los comandos de instalación para cada uno de ellos:


1. Necesitamos tener instalado python3.7 para correr los cálculos de DCA a través del paquete pyDCA

In [ ]:
!apt-get install python3.7 python3-pip python3.7-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

2. También necesitamos HMMER, un programa que permite realizar rápidos alineamientos de perfil

In [ ]:
!apt-get install hmmer

3. Tambien necesitaremos **biopython** and **ipykernel** para nuestro python3.7 para poder instalar **pydca** , mientras que necesitamos **py3Dmol**, **pydca** y **biopython** simultáneamente para correr en nuestro ambiente interactivo de python3.10, todos disponibles a través de `pip`.

In [ ]:
!pip install biopython ipykernel pydca
!python3.10 -m pip install biopython py3Dmol

4. Finalmente instalamos la  **versión de GROMACS complementada con SBM** de los desarrolladores del [servidor SMOG](https://smog-server.org/SBMextension.html), que ha sido precompilada para los propósitos de este tutorial.



In [ ]:
# Descargar y descomprimir SBM-Gromacs
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs_sbm.tar.gz
!tar xzf gromacs_sbm.tar.gz

Una vez que la instalación de estos softwares está completada, estamos listos para realizar los experimentos.

#Parte I – Examinar la estructura de la miohemeritrina de *T. hennahi* myohemerythrin


Primero, descargaremos y visualizaremos la estructura resuelta de la miohemeritrina de *T. hennahi* (PDB 2mhr) utilizando biopython y py3Dmol.

Cuando trabajamos con estructuras de proteínas resueltas por cristalografía de rayos X, la estructura a menudo incluye moléculas de agua cristalinas y también puede contener sustratos o iones. Utilizaremos biopython para limpiar la estructura PDB de átomos no relacionados con proteínas.


1. Descarga la estructura de miohemeritrina de *T. hennahi*  (PDB 2mhr) usando biopython:

In [ ]:
# Importación de tu archivo PDB utilizando biopython
import os
from Bio.PDB import *

# Aquí, agregamos un identificador único o varios IDs de acceso PDB
pdbid = ['2mhr']

# Trataremos los IDs como una lista para descargar todos los PDB
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")
print("¡HECHO!")


2. Limpiando la estructura de la proteina, sacando las moleculas de agua y otros átomos no proteicos usando biopython:

In [ ]:
# Aquí configuramos un analizador para nuestro PDB
parser = PDBParser()
io = PDBIO()
structure = parser.get_structure('A', '2mhr.pdb')

# Ahora eliminamos hidrógenos, aguas y ligandos usando Dice
# y guardamos la estructura limpia con un nombre de archivo diferente
io.set_structure(structure)
sel = Dice.ChainSelector('A', 1, 118)
io.save("2mhr_A.pdb", sel)
print("Tu archivo PDB ha sido procesado. Solo se han conservado los átomos pesados de la proteína.")

Por último, visualizaremos la estructura tridimensional de la miohemeritrina de *T. hennahi* utilizando py3Dmol.

**NOTA❗️** Para otros esquemas de color, consulta los disponibles en el sitio web de [3Dmol](https://3dmol.csb.pitt.edu/doc/types.html#ColorschemeSpec).


In [ ]:
import py3Dmol

# Primero asignamos py3Dmol.view como vista
view = py3Dmol.view()

# Las siguientes líneas se utilizan para agregar la clase addModel
# para leer los archivos PDB
view.addModel(open('2mhr_A.pdb', 'r').read(), 'pdb')

# Aquí configuramos el color de fondo como blanco
view.setBackgroundColor('white')

# Configuramos el estilo y color de la visualización
view.setStyle({'chain': 'A'}, {'cartoon': {'colorscheme': 'ssJmol'}})

# Puedes activar las etiquetas para cada residuo si lo deseas
# o comentarlas con un '#' al principio de cada línea si no lo deseas
view.addResLabels({'resi': '1'}, {'fontColor': 'white', 'fontOpacity': 1, 'showBackground': 'true'})
view.addResLabels({'resi': '118'}, {'fontColor': 'white', 'fontOpacity': 1, 'showBackground': 'true'})

# Centramos la molécula para su visualización
view.zoomTo()

# Finalmente, visualizamos las estructuras con el comando a continuación
view.show()


**PREGUNTAS❓**
- ¿Cuántas hélices tiene esta estructura?
- ¿Cuál es el orden topológico de los elementos de estructura secundaria en la estructura tridimensional?
- ¿Hay alguna región de la proteína que podríamos identificar intuitivamente como desordenada?


#Parte II – Generando un alineamiento múltiple de secuencia (MSA) usando HMMER

Como se ilustra en la FIGURA 1, se requiere una alineación de secuencias múltiples (MSA) de muchas secuencias de una familia de proteínas para inferir interacciones físicas. Una pregunta natural que surge es dónde obtener dichas secuencias.

Como se indicó anteriormente, la miohemeritrina de *T. hennahi* es miembro de la **familia Hemerythrin** en la base de datos **Pfam** [**PF01814**](https://pfam.xfam.org/family/PF01814). La ventaja de Pfam es que contiene **alineamientos iniciales (seed alignments)**, es decir, MSA pequeñas basadas en un subconjunto de secuencias conocidas que se utilizan para **construir perfiles HMM** para buscar homólogos de proteínas u obtener un MSA más grande, así como **MSAs fácilmente disponibles de secuencias identificadas utilizando estos perfiles HMM**.

Utilizaremos estos alineamientos iniciales y las secuencias conocidas de la familia Hemerythrin como entradas para generar un MSA basado en HMM utilizando HMMER.


1. Primero crearemos una carpeta (**_dca_analysis_**) en la que guardaremos todos los archivos de entrada y salida de nuestro análisis coevolutivo.

In [ ]:
# Creemos una carpeta para el análisis coevolutivo
# Necesitamos importar las bibliotecas os y path
import os
from pathlib import Path

# Luego, definimos la ruta de la carpeta que queremos crear.
# Ten en cuenta que la carpeta HOME para una ejecución en Colab es /content/
dcapath = Path("/content/dca_analysis/")

# Ahora, creamos la carpeta utilizando el comando os.mkdir()
# La condición 'if' es simplemente para verificar si la carpeta ya existe
# En caso afirmativo, Python devuelve un error
if os.path.exists(dcapath):
  print("La ruta ya existe")
if not os.path.exists(dcapath):
  os.mkdir(dcapath)
  print("La ruta se creó correctamente")

# Finalmente, nos movemos a esta nueva carpeta
os.chdir(dcapath)

2. Ahora descargaremos la secuencia de la miohemeritrina de *T. hennahi* (PDB 2mhr) en formato FASTA utilizando biopython.

**NOTA❗️** Dado que los archivos PDB pueden contener más de una cadena de polipéptido, es necesario especificar el ID de la cadena desde la cual se extraerá la secuencia de proteínas. En nuestro caso, 2MHR solo contiene una cadena única A, por lo que nuestro ID de acceso es *2MHR_A*.


In [ ]:
import os
from pathlib import Path
from Bio import SeqIO, Entrez

seqlist = ['2MHR_A']

for n in seqlist:
  # Creando una carpeta para nuestra secuencia
  if not os.path.exists(n):
    os.mkdir(n)

  folder = Path(n)

  # Configurando tu correo electrónico para poder usar Entrez
  Entrez.email = 'tu.correo@uc.cl'

  # Aquí, configuramos un manejador temporal con nuestra secuencia descargada en formato fasta
  temp = Entrez.efetch(db="protein", rettype="fasta", id=n)

  # Creando un archivo fasta para escribir nuestra secuencia descargada
  aaseq_out = open(folder / "target.fasta", 'w')

  # Leyendo la información de la secuencia como una cadena en formato fasta
  aaseq = SeqIO.read(temp, format="fasta")

  # Escribiendo el registro de la secuencia en formato fasta
  SeqIO.write(aaseq, aaseq_out, "fasta")

  # Cerrando tanto el manejador temporal como el archivo FASTA
  temp.close()
  aaseq_out.close()


3. Luego, iremos a la [base de datos Pfam](https://pfam.xfam.org) y buscaremos la familia de proteínas adecuada. Puedes hacer esto fácilmente ingresando **_Hemerythrin_** o **_PF01814_** en el cuadro de búsqueda **_Jump To_**. Una vez aquí, obtendremos dos archivos diferentes. El primero es el **alineamiento inicial (seed alignment)**.

- Ve a **_Alignment_ → Format an alignment** y **descarga el alineamiento inicial (seed alignment)**, que se utiliza para construir un perfil HMM. Descarga este alineamiento inicial en formato STOCKHOLM o FASTA.

  Aquí, simplificamos este ejercicio descargando directamente el alineamiento inicial en Google Colab como un archivo de texto (*PF01814_seed*) y utilizando `hmmbuild` de HMMER para generar nuestro perfil HMM.


In [ ]:
!wget -O PF01814_seed.sto.gz https://www.ebi.ac.uk/interpro/wwwapi/entry/pfam/PF01814/\?annotation\=alignment\:seed
!gunzip -d PF01814_seed.sto.gz
!hmmbuild GA.cm PF01814_seed.sto

- El segundo archivo es **un conjunto de secuencias no alineadas de una de las bases de datos disponibles** en formato FASTA, asegurándote de seleccionar la opción **`no gaps (unaligned)`** en la sección **GAPS**. Agregaremos nuestra secuencia objetivo a este conjunto de secuencias (por eso estamos utilizando un formato FASTA) y generaremos una MSA utilizando nuestro perfil HMM generado y HMMER.

  Aunque cuantas más secuencias, mejor, **recomendamos encarecidamente descargar las secuencias de RP15, RP35 o RP55** debido a nuestras limitaciones de tiempo.

  Una vez descargado, importa este archivo a Google Colab arrastrándolo a la carpeta correspondiente en la pestaña **Files**, ubicada en la barra lateral izquierda de Google Colab, y rebautízalo como **_PF01814_seqs.txt_**.


**⚠️ ADVERTENCIA:** El sitio de Pfam se ha fusionado en Interpro, por lo que solo podemos seleccionar descargar el alineamiento inicial (seed) o la secuencia completa en formato multifasta. Para este ejercicio, utilizaremos la lista completa de secuencias.


In [ ]:
!wget -O hemerythrin_interpro.sto.gz https://www.ebi.ac.uk/interpro/wwwapi/entry/pfam/PF01814/\?annotation=alignment:full
!gunzip -d hemerythrin_interpro.sto.gz

4. A continuación, convertimos nuestro alineamiento al formato FASTA para tener más control sobre las secuencias dentro de él. Hacemos esto utilizando el script reformat.pl de HHSuite. Su uso es el siguiente:




```
reformat.pl [informat] [outformat] infile outfile [options]
```



In [ ]:
!wget -O /content/reformat.pl https://raw.githubusercontent.com/soedinglab/hh-suite/master/scripts/reformat.pl
!perl /content/reformat.pl sto fas hemerythrin_interpro.sto hem_alignment.fas

#Necesitamos desalinear nuestras secuencias para que podamos incorporar la que nos interesa
!sed -i 's/-//g' hem_alignment.fas

5. Luego, agregaremos nuestra secuencia objetivo a nuestro conjunto de secuencias y utilizaremos el perfil HMM para generar una MSA utilizando `hmmalign` de HMMER. La MSA resultante se almacenará como **_PF01814_2mhr_aligned.afa_**.


In [ ]:
!cat 2MHR_A/target.fasta hem_alignment.fas > multifasta.fasta
!hmmalign -o PF01814_2mhr_aligned.afa --outformat AFA GA.cm multifasta.fasta

Luego, simplemente verificamos que la primera secuencia del alineamiento coincida con nuestra secuencia objetivo:


In [ ]:
!head PF01814_2mhr_aligned.afa

# Parte III - Inferir interacciones físicas entre pares de residuos en un MSA mediante análisis coevolutivo en pyDCA


La MSA resultante será nuestra entrada para nuestro análisis coevolutivo utilizando la **aproximación de campo medio de DCA (mfDCA)**, uno de los dos enfoques disponibles (el otro es la pseudo-máxima verosimilitud o plmDCA) en pyDCA, siendo mfDCA más rápido de calcular.

pyDCA es una implementación en Python de DCA que no solo infiere contactos físicos a partir de la MSA, sino que también asegura la precisión de nuestra predicción estableciendo el número de verdaderos positivos en comparación con los contactos observados en una estructura de proteína resuelta.

En resumen, DCA permitirá desentrañar las contribuciones directas a las correlaciones (resultantes de contactos nativos) de las contribuciones indirectas (mediadas a través de cadenas de contactos nativos). Dado que nuestra intención es predecir la estructura de la miohemeritrina de *T. hennahi* basándonos únicamente en la información de la secuencia, utilizaremos su secuencia de proteína como referencia para la predicción de contactos entre pares de residuos.

**NOTA❗️** Si bien los algoritmos de física estadística disponibles en pyDCA están fuera del alcance de este tutorial, se puede encontrar más información sobre mfDCA y plmDCA en:
- Ekeberg, M., Lövkvist, C., Lan, Y., Weigt, M., & Aurell, E. (2013). Improved contact prediction in proteins: using pseudolikelihoods to infer Potts models. Physical Review E, 87(1), 012707.


1. Luego, recortaremos nuestro archivo MSA según la longitud de la secuencia objetivo, que en este caso corresponde a la secuencia de la miohemeritrina de *T. hennahi* que fue resuelta por cristalografía de rayos X (2JWS). El código a ejecutar es el siguiente:


 `pydca trim_by_refseq <biomolecule>  <alignment.fa>  <refseq_file.fa> --remove_all_gaps --verbose`

 Esto comparará nuestra secuencia objetivo con todas las secuencias en el alineamiento. Posteriormente, seleccionará la más cercana y recortará todas las posiciones de huecos. La MSA resultante se generará en una carpeta nombrada según el archivo con el prefijo "Trimmed".


In [ ]:
!pydca trim_by_refseq protein  PF01814_2mhr_aligned.afa  2MHR_A/target.fasta --remove_all_gaps --verbose

Ahora podemos revisar que la primera secuencia esté correctamente recortada.

In [ ]:
!head Trimmed_PF01814_2mhr_aligned/Trimmed_PF01814_2mhr_aligned.fa

3. Una vez hecho esto, calcularemos nuestros puntajes DCA utilizando el **enfoque de campo medio (mfDCA)**. Para este análisis, utilizamos el programa 'mfdca' instalado para analizar nuestra MSA recortada. También proporcionamos una serie de parámetros opcionales:
- Un pseudocuento, que básicamente agrega observaciones "falsas" adicionales a los datos reales para curar singularidades causadas por correlaciones fuertes en datos insuficientemente muestreados, utilizando el parámetro `pseudocount`. Este valor suele establecerse en un tercio (0.3) a la mitad (0.5) del número total de secuencias en la MSA.
- APC es el puntaje que obtendremos del análisis, corresponde a la norma de Frobenius de la matriz de acoplamiento entre los sitios $i$ y $j$, que luego se corrige por el promedio del producto.


In [ ]:
!mfdca compute_fn protein Trimmed_PF01814_2mhr_aligned/Trimmed_PF01814_2mhr_aligned.fa --apc --pseudocount 0.5 --verbose

4. Una vez que se completen estos cálculos de mfDCA, podemos imprimirlos en la pantalla. Aquí, utilizaremos Python para imprimir los 10 pares principales de DCA.


In [ ]:
!head -n 22 MFDCA_output_Trimmed_PF01814_2mhr_aligned/MFDCA_apc_fn_scores_Trimmed_PF01814_2mhr_aligned.txt

5. Una forma más significativa de determinar cuán buenas son nuestras predicciones de contactos físicos es compararlas visual y estadísticamente con datos experimentales.

  Para este propósito, generaremos un `DCAVisualizer` para comparar nuestro mapa de contactos inferido con el experimental derivado de la estructura de proteína de la miohemeritrina de *T. hennahi*.

  Este visualizador toma las siguientes entradas:




```
pydca plot_contact_map <biomolecule> <PDB_chain_name> <PDB_id/PDB_file.PDB> <refseq.fa> <DCA_file.txt> --verbose  
```



In [ ]:
!pydca plot_contact_map protein A 2MHR 2MHR_A/target.fasta\
 MFDCA_output_Trimmed_PF01814_2mhr_aligned/MFDCA_apc_fn_scores_Trimmed_PF01814_2mhr_aligned.txt --verbose

6. Ahora trazaremos ambos mapas de contactos, con los contactos basados en la estructura en el triángulo superior izquierdo y los contactos basados en DCA en el triángulo inferior derecho. Los contactos correctamente predichos se muestran en verde, mientras que los falsos positivos se muestran en rojo. Además, ten en cuenta que el número de contactos DCA es equivalente a $L$, donde $L$ es la longitud (es decir, el número de columnas) del alineamiento recortado.


In [ ]:
!awk 'NR>=23{print $5, $6, $1}' /content/dca_analysis/contact_map_2MHR/contact_map2MHR.txt > contacts.txt
!echo X Y Z > /content/headers
!cat /content/headers /content/dca_analysis/contacts.txt > /content/contacts2.txt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

data = np.genfromtxt('/content/contacts2.txt', dtype=None, names=True, encoding=None)
x = data['X']
y = data['Y']
z = data['Z']

category_colors = {
    'tp': 'green',
    'fp': 'red',
    'pdb': 'gray'
}

inverted_x = []
inverted_y = []
colors = []

plt.figure(figsize=(6, 6))

for i in range(len(z)):
    category = z[i]
    if category in ('tp', 'fp'):
        inverted_x.append(y[i])
        inverted_y.append(x[i])
        colors.append(category_colors[category])
    else:
        inverted_x.append(x[i])
        inverted_y.append(y[i])
        colors.append(category_colors[category])

plt.scatter(inverted_x, inverted_y, c=colors, marker='o', s=5)
for category, color in category_colors.items():
    plt.scatter([], [], c=color, label=category)

plt.xlabel('Residue index')
plt.ylabel('Residue index')
plt.title('Contact map for the mfDCA prediction')
plt.legend()
plt.grid(True)
plt.show()

7. Por último, determinaremos la precisión de nuestro análisis coevolutivo observando su tasa de verdaderos positivos (TP) por rango. La tasa de TP por rango es el número de contactos correctamente predichos por rango de los pares predichos dividido por todas las predicciones en ese rango.

  En el gráfico resultante, la línea azul corresponderá a las tasas de TP para los contactos predichos, y la línea naranja a la tasa teóricamente máxima posible de verdaderos positivos para los contactos obtenidos de la estructura experimental.


In [ ]:
!pydca plot_tp_rate protein A 2MHR 2MHR_A/target.fasta \
MFDCA_output_Trimmed_PF01814_2mhr_aligned/MFDCA_apc_fn_scores_Trimmed_PF01814_2mhr_aligned.txt

Ahora graficamos la tasa de TP

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
data = pd.read_csv('/content/dca_analysis/TPR_2MHR/TPR_2MHR.txt', delimiter='\t', skiprows=11, header=None, usecols=[0])

data['Index'] = range(1, len(data) + 1)
data.set_index('Index', inplace=True)
plt.semilogx(data.index, data[0], linestyle='-')
plt.xlabel("Rank")
plt.ylabel("TPR")
plt.title("True positive rate for sorted APC")
plt.show()

**PREGUNTAS❓**
- ¿Cómo cambiaría el número de contactos verdaderos positivos en función del número de secuencias en la MSA?
- ¿Cómo cambiaría el número de contactos verdaderos positivos en función de la identidad de secuencia en la MSA?


# Parte IV - Generar un SBM basado en DCA y en información de estructura secundaria para la predicción de la estructura de proteínas


Los SBM se basan en el tratamiento de **contactos nativos**, es decir, pares de residuos que caen por debajo de un umbral de distancia en una estructura proteica dada, como **interacciones no enlazadas atractivas a través de potenciales de Lennard-Jones o Gaussianos**, mientras que todas las demás interacciones no nativas se tratan como repulsivas.

En este contexto, **el uso de interacciones físicas entre pares de residuos inferidos por DCA** en lugar de contactos nativos definidos experimentalmente es una extensión natural a SBM, debido a la simplicidad de su componente atractivo en la función de energía.

Aquí, utilizaremos nuestros contactos predichos, junto con potenciales enlazados (enlaces, ángulos y dihedros) dibujados en función de las predicciones de estructura secundaria utilizando **Jpred4**, para generar un SBM que permita simulaciones de plegamiento de proteínas para predecir la estructura de la miohemeritrina de *T. hennahi* basada únicamente en la información de secuencia.

**NOTA❗️** Si bien los detalles de la predicción de estructuras secundarias basada en secuencias están fuera del alcance de este tutorial, una buena introducción se encuentra en la siguiente publicación:

- Rost, B. (2001). Protein secondary structure prediction continues to rise. Journal of structural biology, 134(2-3), 204-218.


1. Comenzaremos creando una nueva carpeta en la que generaremos nuestro SBM para la miohemeritrina de *T. hennahi*.


In [ ]:
# Creemos una carpeta para crear nuestro modelo basado en estructuras (SBM)
sbmpath = Path("/content/sbm/")

if os.path.exists(sbmpath):
  print("la ruta ya existe")
if not os.path.exists(sbmpath):
  os.mkdir(sbmpath)
  print("la ruta se creó con éxito")

# Cambiando al directorio de la carpeta sbm
os.chdir(sbmpath)

# También copiaremos el archivo PDB limpio de 2MHR
!cp ../2mhr_A.pdb experimental.pdb

2. Ahora, almacenaremos los primeros $L$ contactos de DCA, donde $L$ es la longitud del (es decir, el número de columnas) de la MSA, como un archivo de texto de 3 columnas con los pares de residuos en las columnas 1 y 2 y los puntajes de DCA en la columna 3.

  Estos contactos se utilizarán para definir **interacciones no enlazadas atractivas entre pares de residuos** en nuestro SBM.

In [ ]:
# Establecer la longitud L, equivalente a 'Número de contactos DCA' en nuestras gráficas anteriores
%%bash
Longitud=118
awk 'NR>12&&$1+2<$2{print $1, $2, $3}' /content/dca_analysis/MFDCA_output_Trimmed_PF01814_2mhr_aligned/MFDCA_apc_fn_scores_Trimmed_PF01814_2mhr_aligned.txt | head -n $Longitud
awk 'NR>12&&$1+2<$2{print $1, $2, $3}' /content/dca_analysis/MFDCA_output_Trimmed_PF01814_2mhr_aligned/MFDCA_apc_fn_scores_Trimmed_PF01814_2mhr_aligned.txt | head -n $Longitud > predicted_contact_map.txt

3. También necesitaremos una predicción de la estructura secundaria para generar las **interacciones enlazadas** de nuestro SBM. Para ello, visitaremos el [servidor web Jpred4](http://www.compbio.dundee.ac.uk/jpred/index.html), introduciremos nuestra secuencia en el cuadro de texto y haremos clic en **`Make a Prediction`**, confirmaremos que aún queremos hacer una predicción haciendo clic en **`Continue`** y luego seleccionaremos **`View simple results in HTML`**.

  Luego, carga las primeras y segundas líneas de los resultados obtenidos por Jpred4 en el siguiente cuadro de texto.

In [ ]:
#@title Ingresa la secuencia de aminoácidos y la estructura secundaria predicha de tu proteína
sequence = ''  #@param {type:"string"}
Jpred = ''  #@param {type:"string"}
f = open("Jpred.txt", "a")
f.write(sequence+"\n")
f.write(Jpred)
f.close()

4. Los contactos terciarios inferidos por DCA y la estructura secundaria predicha por Jpred4 se utilizarán ahora para generar nuestro SBM mediante el siguiente script. El resultado de este script es:
- Un archivo de **coordenadas** (extensión **.gro**), donde cada residuo de la secuencia objetivo está representado por una única cuenta y se le asignan posiciones iniciales para comenzar nuestras simulaciones de plegamiento.
- Un archivo de **topología** (extensión **.top**), que contiene las interacciones no enlazadas de los contactos DCA como potenciales gaussianos y las interacciones enlazadas (enlaces, ángulos, dihedros) extraídas de la predicción de la estructura secundaria como potenciales armónicos.

  Al ejecutar este script, se te pedirá que indiques un `factor de fuerza máximo de DCA`, que controla la fuerza de los potenciales pairwise que impulsarán el plegamiento de la proteína. Una buena estimación para la fuerza de las interacciones coevolutivas es considerar el factor de fuerza equivalente a la razón de $L$ sobre el número de pares de DCA (factor = $L$/$|$DCA$|$). Así que **usaremos un valor de 1.**


In [ ]:
#Descarga de todos los archivos requeridos
!wget https://github.com/pb3lab/ibm3202/raw/master/scripts/dcasbm.py
!wget https://github.com/pb3lab/ibm3202/raw/master/scripts/distavg
!wget https://github.com/pb3lab/ibm3202/raw/master/files/sbm_calpha_SA_v5_short.mdp

In [ ]:
!python dcasbm.py Jpred.txt predicted_contact_map.txt

5. Por último, utilizaremos nuestros archivos de coordenadas y topología junto con un archivo de parámetros para configurar y ejecutar una simulación de plegamiento en nuestra versión mejorada de GROMACS con SBM.

  Estas simulaciones comienzan a una temperatura alta en la que la proteína está desplegada, y luego la temperatura del sistema se reduce gradualmente hasta que la proteína alcanza un mínimo de energía donde la mayoría de las interacciones están satisfechas.


**⚠️ADVERTENCIA:** Existe un problema no resuelto con el uso de potenciales no enlazados gaussianos en la versión mejorada de GROMACS con SBM en Google Colab, lo que provoca errores en las simulaciones al ejecutar durante largos períodos de tiempo. Por lo general, esto se reflejará en el archivo run.gro mostrando **`nan`** en lugar de posiciones de coordenadas para todos los átomos. Por lo tanto, nos limitamos a simulaciones muy cortas en este tutorial. Si encuentra este error, intente repetir sus simulaciones. **¡Proceda con precaución!**


In [ ]:
%%time
%%bash
source /content/gromacs_sbm/bin/GMXRC
#precompilando los archivos para la simulación
gmx grompp -f sbm_calpha_SA_v5_short.mdp -c Jpred_calpha.gro -p Jpred_calpha.top -o run.tpr
#corriendo la simulación de plegado
gmx mdrun -deffnm run -nt 1 -noddcheck

**⚠️ ¡COPIA DE SEGURIDAD DE EMERGENCIA!** ¿Falló tu simulación? ¡Usa la siguiente celda de código para descargar una ejecución de simulación exitosa!


In [ ]:
#Solo en caso de emergencia
#!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/emergency_backup/lab13/dca_sbm_backup.tar.gz
#!tar xzf dca_sbm_backup.tar.gz

6. Convertiremos ahora nuestra estructura final en formato .gro a un archivo PDB para su posterior comparación con la estructura experimental de la miohemeritrina de *T. hennahi*.


In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#Convirtiendo la estructura desde un .gro a un PDB
gmx editconf -f run.gro -o predicted.pdb

7. Una vez que nuestra simulación de plegamiento esté completa y hayamos convertido nuestras coordenadas finales al formato de archivo PDB, podemos comparar la estructura predicha basada únicamente en la información de la secuencia con la estructura determinada experimentalmente de la miohemeritrina de *T. hennahi*. Para esto, primero realizaremos una alineación estructural de ambas estructuras.

   En el siguiente cuadro, por favor ingrese el nombre de archivo de la estructura experimental (incluyendo su extensión .pdb) e indique los residuos de inicio y fin que desea alinear (por ejemplo, 1 a 118).

   **NOTA❗️** Recuerde que, típicamente, los extremos N- y C-terminales de una estructura proteica son muy flexibles y pueden ser ignorados en la alineación estructural para obtener mejores resultados.


In [ ]:
#@title Realiza un alineamiento estructural

import Bio.PDB

# Select what residues numbers you wish to align
# and put them in a list

#@markdown 1. Indica los residuos iniciales y finales que deseas alinear

start_resi = 17 #@param {type:"number"}
end_resi   = 118 #@param {type:"number"}
atoms_to_be_aligned = range(start_resi, end_resi + 1)

# Start the parser
pdb_parser = Bio.PDB.PDBParser(QUIET = True)

#@markdown 2.Indica el nombre de la estructura experimental (Archivo PDB)

# Get the structures
experimental = 'experimental.pdb' #@param {type:"string"}
predicted = 'predicted.pdb'
ref_structure = pdb_parser.get_structure("reference", experimental)
sample_structure = pdb_parser.get_structure("sample", predicted)

# Use the first model in the pdb-files for alignment
# Change the number 0 if you want to align to another structure
ref_model    = ref_structure[0]
sample_model = sample_structure[0]

# Make a list of the atoms (in the structures) you wish to align.
# In this case we use CA atoms whose index is in the specified range
ref_atoms = []
sample_atoms = []

# Iterate of all chains in the model in order to find all residues
for ref_chain in ref_model:
  # Iterate of all residues in each model in order to find proper atoms
  for ref_res in ref_chain:
    # Check if residue number ( .get_id() ) is in the list
    if ref_res.get_id()[1] in atoms_to_be_aligned:
      # Append CA atom to list
      ref_atoms.append(ref_res['CA'])

# Do the same for the sample structure
for sample_chain in sample_model:
  for sample_res in sample_chain:
    if sample_res.get_id()[1] in atoms_to_be_aligned:
      sample_atoms.append(sample_res['CA'])

# Now we initiate the superimposer:
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(ref_atoms, sample_atoms)
super_imposer.apply(sample_model.get_atoms())

# Print RMSD:
print('The calculated RMSD is:')
print (super_imposer.rms)

# Save the aligned version of one of the chains of 6ANE
io = Bio.PDB.PDBIO()
io.set_structure(sample_structure)
io.save("predicted_aligned.pdb")

aligned = 'predicted_aligned.pdb'

7. Ahora podemos cargar la estructura superpuesta en py3dmol con la celda de código que se muestra a continuación para visualizar la precisión de nuestra predicción.

   **PREGUNTA❓**
   - ¿Qué tan precisa fue tu predicción en comparación con la estructura resuelta experimentalmente de la miohemeritrina de *T. hennahi*?
   - ¿De qué maneras crees que se podría mejorar la precisión de tu predicción?


In [ ]:
view=py3Dmol.view()
view.addModel(open(experimental, 'r').read(),'pdb')
view.setStyle({'chain':'A'},{'cartoon': {'color':'spectrum'}})
view.addModel(open(aligned, 'r').read(),'pdb')
view.setStyle({'chain':' '},{'cartoon': {'style':'trace','color':'red'}})
#Puedes borrar las siguientes etiquetas
view.addResLabels({'resi':'1'},{'fontColor':'white','fontOpacity':1,'showBackground':'true'})
view.addResLabels({'resi':'118'},{'fontColor':'white','fontOpacity':1,'showBackground':'true'})

view.zoomTo()
view.setBackgroundColor('white')
view.show()